# Smarte interaktive Skripte (smartiS)

[P. Kersten](https://www.hshl.de/personen/prof-dr-ing-peter-kersten), J. Jeschonowski, L.P. Kamegne-Kamdem, N.B. Neathery und D.M. Weis

[Hochschule Hamm-Lippstadt](https://www.hshl.de), Marker Allee 76–78, D-59063 Hamm

## Stoßsimulation
- Geschwindigkeit nach einem inelastischen Stoß 
- Spezialfall: Inelastischer Stoß zweier Körper mit gleicher Masse und gleicher Geschwindigkeit
- Geschwindigkeiten nach einem elastischen Stoß
- Spezialfall: Elastischer Stoß zweier Körper mit gleicher Masse und einem ruhenden Körper




In [1]:
%matplotlib inline
import numpy as np
import sympy as sp
import math as m
import matplotlib.pyplot as plt
import ipywidgets as widgets
import scipy.optimize
from ipywidgets import interact
from sympy import symbols
from sympy.plotting import plot3d
from matplotlib.pyplot import figure
from scipy.optimize import curve_fit
from IPython.display import Markdown, clear_output, display, HTML

In [2]:
javascript_funktion = {False: "hide()", True: "show()"}
schaltflaeche  = {False: "Quellcode anzeigen", True: "Quellcode verbergen"}


def quellcode_umschalten(state):

    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_funktion[state],)
    output        = output_string.format(*output_args)
    display(HTML(output))


def schaltflaeche_aktion(value):

    state = value.new
    quellcode_umschalten(state)
    value.owner.description = schaltflaeche[state]


state = False
quellcode_umschalten(state)

button = widgets.ToggleButton(state, description = schaltflaeche[state])
button.observe(schaltflaeche_aktion, "value")

display(button)

ToggleButton(value=False, description='Quellcode anzeigen')

## Stoßsimulation

Probieren Sie selbst aus und sehen Sie was passiert. Stellen Sie die Spezialfälle nach. Überlegen Sie sich was nach dem Stoß passiert und drücken Sie dann erst den zweiten Playbutton!<br>
**WICHTIG:**<br>
Stelle die gewünschten Parameter ein. Klicke dann auf den **ersten Playbutton**. Sobald die Animation stoppt, drücke auf den **dazugehörigen Pausebutton**. Nun kannst du den **zweiten Playbutton** drücken. Um die Animation ein zweites mal zu starten, drücke **beide Stopbuttons**! Danach kannst du die Schritte wiederholen.

In [3]:
from smartiS import *
import ipywidgets as widgets
import sympy as sp
import numpy as np

from math import pi, sin, cos
from ipywidgets import HBox, VBox, Layout

class Impuls(smartiS):
    
    def __init__(self):
        
        # Ebenen definieren
        layers = ["masse1"]
        lx = 750
        ly = 650

        # smartiS-Klasse initialisieren
        super().__init__(canvases = layers,
                         width    = lx,
                         height   = ly)
        
        # Elemente erzeugen
       
        self.v1_slider = widgets.IntSlider(min = -10,
                                          max = 10,
                                          value = 5,
                                          description = r"$v1$")
        self.v1_slider.observe(self.draw_masse1, "value")

        
        self.m1_slider = widgets.FloatSlider(min = 1.0,
                                          max = 250.0,
                                            step=0.1,
                                          value = 2.0,
                                          description = r"$m1$")
        self.m1_slider.observe(self.draw_masse1, "value")
        
        self.m2_slider = widgets.FloatSlider(min = 1.0,
                                          max = 250.0,
                                           step=0.1,
                                          value = 2.0,
                                          description = r"$m2$")
        self.m2_slider.observe(self.draw_masse1, "value")
        
        self.v2_slider = widgets.IntSlider(min = -10,
                                          max = 10,
                                          value = -5,
                                          description = r"$v2$")
        self.v2_slider.observe(self.draw_masse1, "value")
        
        self.time1_slider = widgets.FloatSlider(min = 0.0,
                                               max = 500.0,
                                               step = 0.1,
                                               value = 0.0)
        
        self.time2_slider = widgets.FloatSlider(min = 0.0,
                                               max = 500.0,
                                               step = 0.1,
                                               value = 0.0)
        
        self.play1 = widgets.Play(
            value=0,
            min=0,
            max=5000,
            step=1,
            interval=25,
            description="Starten",
            disabled=False)
        self.play1.observe(self.draw_masse1, "value")
        widgets.jslink((self.play1, 'value'), (self.time1_slider, 'value'))
        
        self.play2 = widgets.Play(
            value=0,
            min=0,
            max=5000,
            step=1,
            interval=25,
            description="Weiterführend",
            disabled=False)
        self.play2.observe(self.draw_masse1, "value")
        widgets.jslink((self.play2, 'value'), (self.time2_slider, 'value'))

        self.auswahl_buttons = widgets.RadioButtons(options=['inelastisch', 'elastisch'],
                                                    value='elastisch',
                                                    layout={'width': 'max-content'},
                                                    description='Stoßart:',
                                                    disabled=False
                                                   )
        self.auswahl_buttons.observe(self.draw_masse1, "value")
        

        # Elemente darstellen
        display(HBox([
            self.canvas,
            VBox([self.auswahl_buttons,
                  self.m1_slider,
                  self.m2_slider,
                  self.v1_slider,
                  self.v2_slider,
                  self.play1,
                  self.play2,
                 ])
        ]))
        
        self.draw_masse1()
    
    
    
    def draw_masse1(self,*args):
        
        # Masse1 zeichnen
        masse1 = self["masse1"]
        a = self.auswahl_buttons.value
        tA = self.time1_slider.value
        tE = self.time2_slider.value
        
        m1 = self.m1_slider.value
        v1A = self.v1_slider.value
        d1 = ((6 * m1)/pi)**(1/3) * 5
        x1 = 250 + v1A * tA
        
        m2 = self.m2_slider.value
        v2A = self.v2_slider.value
        d2 = ((6 * m2)/pi)**(1/3) * 5
        x2 = 500 + v2A * tA        
            
        #Eine Funktion, die den Treffzeitpunkt berechnet
        t_T = (500-250)/(v1A-v2A)
        
        #Treffzeitpunkt in Anfang Bewegungsgleichung setzen und x1T und x2T berechnen
        x1T = (250 + v1A * t_T) - d1
        x2T = (500 + v2A * t_T) + d2
        

            #if statemant mit Radiobutton elastischer und inelastischer Stoß
        if a == 'inelastisch':
            vE = m1/(m1+m2) * v1A + m2/(m1+m2) * (v2A)
            x1E = x1T + (vE * tE)
            x2E = x2T + (vE * tE)
            
        elif a == 'elastisch':
            v1E = (m1 * v1A + m2 * (2*(v2A) - v1A))/(m1 + m2)
            v2E = (m2 * (v2A) + m1 * (2*v1A - (v2A)))/(m1 + m2)
            x1E = x1T + (v1E * tE)
            x2E = x2T + (v2E * tE)
            
        #vor dem Stoß
        if (x1 + (d1)) < (x2 - (d2)):
            
            with hold_canvas(masse1):

                masse1.clear()
                masse1.stroke_arc(x1, 325, d1, 0, 2*pi)
                masse1.fill_style = "#00ffff"
                masse1.fill_arc(x1, 325, d1, 0, 2*pi)

                masse1.stroke_arc(x2, 325, d2, 0, 2*pi)
                masse1.fill_style = "#ffff00"
                masse1.fill_arc(x2, 325, d2, 0, 2*pi)

            
        #nach dem Stoß
        elif (x1 + (d1)) >= (x2 - (d2)):
            
            with hold_canvas(masse1):

                masse1.clear()
                masse1.stroke_arc(x1E, 325, d1, 0, 2*pi)
                masse1.fill_style = "#00ffff"
                masse1.fill_arc(x1E, 325, d1, 0, 2*pi)

                masse1.stroke_arc(x2E, 325, d2, 0, 2*pi)
                masse1.fill_style = "#ffff00"
                masse1.fill_arc(x2E, 325, d2, 0, 2*pi)
                                
    
            
      
            
I = Impuls()


# Wie fandet ihr dieses smarti?

[Lasst uns gerne Feedback da!](https://pingo.coactum.de/460203) Viel Spaß wünscht das smartiS Team :-)

Dezemeber 2020 (CC BY-SA 4.0)